# Needed Modules

In [49]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
import matplotlib.patches as mpatches

In [1]:
# from huggingface_hub import notebook_login
# from huggingface_hub import upload_file
# from huggingface_hub import delete_file
# from huggingface_hub import Repository
# notebook_login()

# Import Fine-Tuned GeoBERT

In [4]:
from transformers import TFAutoModelForTokenClassification, TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
model_checkpoint = "botryan96/GeoBERT_analyzer"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
models = TFAutoModelForSequenceClassification.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/373 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/201k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/599k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/737 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some layers from the model checkpoint at botryan96/GeoBERT_analyzer were not used when initializing TFBertForSequenceClassification: ['dropout_75']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at botryan96/GeoBERT_analyzer.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


# Data Preparation

In [94]:
sentence1 = ['The green energy problem has been a lovely issue in America and Europe.']
sentence2 = ['Ryan works for some energy industries in America while his love is in Europe.']

In [95]:
outputs1 = tokenizer(sentence1, return_tensors="np", padding="longest")
out1 = models(outputs1, output_hidden_states= True)

outputs2 = tokenizer(sentence2, return_tensors="np", padding="longest")
out2 = models(outputs2, output_hidden_states= True)

In [96]:
loc = outputs1.word_ids()
tokens = outputs1.tokens()
ids = tokenizer.convert_tokens_to_ids(tokens)

print(list(zip(tokens, ids, loc)))
print(' ')

[('[CLS]', 2, None), ('the', 106, 0), ('green', 1699, 1), ('energy', 3726, 2), ('problem', 5844, 3), ('has', 433, 4), ('been', 371, 5), ('a', 23, 6), ('lo', 2882, 7), ('##vel', 540, 7), ('##y', 70, 7), ('issue', 7789, 8), ('in', 117, 9), ('america', 7769, 10), ('and', 119, 11), ('europe', 4243, 12), ('.', 11, 13), ('[SEP]', 3, None)]
 


In [97]:
loc = outputs2.word_ids()
tokens = outputs2.tokens()
ids = tokenizer.convert_tokens_to_ids(tokens)

print(list(zip(tokens, ids, loc)))
print(' ')

[('[CLS]', 2, None), ('ryan', 12315, 0), ('works', 10548, 1), ('for', 162, 2), ('some', 528, 3), ('energy', 3726, 4), ('industries', 10832, 5), ('in', 117, 6), ('america', 7769, 7), ('while', 2453, 8), ('his', 5030, 9), ('lo', 2882, 10), ('##ve', 147, 10), ('is', 151, 11), ('in', 117, 12), ('europe', 4243, 13), ('.', 11, 14), ('[SEP]', 3, None)]
 


In [98]:
tes1 = out1.hidden_states[-1][0]
tes1
col1 = [i for i in range(len(tes1[0]))]
lst1 = tes1.numpy().tolist()
df1 = pd.DataFrame(lst1, columns = col1, dtype = float)
df1

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,0.526861,1.708692,1.125218,0.103458,0.517974,1.270152,-0.889633,-2.128117,-0.580222,-1.673796,...,0.073119,1.389552,1.134918,-0.371036,-1.545351,-1.282610,-0.825831,-0.855169,-1.406805,-1.271665
1,0.559776,0.986373,1.370549,-1.002640,-1.597162,1.568433,0.742627,-1.742769,-0.341264,-1.858063,...,-0.155558,1.190248,2.238168,0.641926,-1.556880,-0.419763,0.117314,0.887583,-1.074012,-1.450861
2,0.409344,2.308864,0.781421,-0.959297,-0.932325,0.570019,0.231656,-1.689713,-0.792942,-0.781275,...,0.805858,1.237293,1.530544,-0.793038,-0.904950,0.379752,-0.518825,-0.459834,-1.784434,-0.374684
3,-0.805968,0.445693,-0.212789,-1.490984,0.074572,-1.259386,0.501055,0.463613,0.342775,-1.467355,...,-0.124837,0.343094,0.325841,0.111106,0.121279,-0.805288,0.629488,-1.848197,-0.311849,-0.144681
4,-1.431995,1.711769,-0.183973,-0.949505,-0.300466,0.790710,0.316157,-0.895452,-0.380088,-2.066049,...,0.831755,0.739650,0.474811,-0.653361,0.021514,0.134313,-0.018569,1.146340,-0.654342,-0.504749
5,-0.207164,1.223704,0.426163,-1.400387,-0.607666,0.571201,-1.492018,-0.940277,-0.850535,-1.555445,...,0.309610,0.813578,1.731485,-0.188817,0.025634,-0.096194,0.246613,-0.588939,-2.394382,-0.831336
6,-0.083866,0.458309,0.257573,-1.050743,0.223076,0.126370,-1.146976,-0.785626,-0.371213,-1.675178,...,-0.136472,1.452729,0.790580,0.364804,-0.507634,-0.507099,1.319732,-1.725207,-1.300440,-0.171617
7,0.763458,0.389486,1.744185,-0.179043,-1.066321,1.421597,-0.219950,-1.490874,-0.695673,-0.420442,...,-0.866535,-0.116628,0.162466,0.862676,-0.747668,-0.329397,1.090864,0.771885,-1.025625,-1.187392
8,0.592406,1.193478,0.786389,-0.491592,-0.984515,0.672686,0.375335,-1.778849,-1.276158,-1.765607,...,0.281838,0.008337,1.107081,0.161767,-1.362094,0.651170,0.249711,-1.055596,-0.919503,-0.836657
9,0.349862,1.422941,0.590019,-0.701456,-0.902058,0.620555,-0.594303,-0.569866,-1.039921,-0.772726,...,0.666599,0.910157,1.329038,-0.076275,-0.852221,-0.054070,0.529129,-1.047296,-0.774852,-1.503151


In [99]:
tes2 = out2.hidden_states[-1][0]
tes2
col2 = [i for i in range(len(tes2[0]))]
lst2 = tes2.numpy().tolist()
df2  = pd.DataFrame(lst2, columns = col2, dtype = float)
df2

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,1.317990,1.303441,0.508046,-1.012868,-0.814912,-0.211947,0.050318,0.396978,-0.099615,0.913923,...,0.383212,-0.507406,0.542911,0.377075,-0.498669,1.202264,0.553271,-0.423616,-0.584370,-1.178537
1,-0.126120,0.945822,0.817309,-0.901786,-1.281811,0.577872,-0.575280,-1.144387,-0.937899,-0.428756,...,0.292237,-0.284725,0.838840,0.008309,-0.901890,0.849643,0.265553,-0.845185,-1.602575,-1.482636
2,-2.060981,1.468702,0.081903,-1.299391,-0.236358,0.832565,1.265005,0.272433,-0.407072,0.295295,...,1.587976,0.177375,0.981494,0.448281,0.715820,0.256130,0.031214,0.344961,-0.710749,-0.391703
3,-0.029254,0.709929,0.950618,-1.347424,-0.724877,0.702199,0.681385,-1.035072,-1.310820,-0.078759,...,0.305372,-0.014974,1.563137,-0.345107,-0.189799,0.456455,0.197201,0.754403,-1.728047,-1.303501
4,0.076310,1.766653,0.984876,-1.496902,-1.042032,1.379063,0.360621,-0.997997,-1.226498,-0.502149,...,-0.276422,-0.301983,1.532265,0.394213,-0.337609,0.880133,0.477662,0.391227,-1.292357,-1.100600
5,-0.745638,0.711231,-0.281942,-1.798130,-0.771951,-0.918206,0.498169,0.236460,-0.539661,0.234051,...,0.460944,0.040657,-0.412405,0.505739,0.203581,0.545334,0.462181,-0.702651,-0.041726,0.058779
6,-0.544005,0.348523,1.406909,-0.797408,-0.356304,-0.000500,0.974728,-0.926920,-0.564035,-1.839472,...,0.027140,-1.101458,0.628224,0.723604,0.642250,0.667115,1.065796,-0.405096,-0.666210,-0.661841
7,0.381857,0.172362,0.782374,-1.815053,-0.463437,0.876038,0.575691,-0.772793,-1.160844,-0.595242,...,-0.169051,-1.440793,0.589112,0.175402,-0.316117,1.461917,-0.020055,0.985393,-0.653783,-1.489531
8,-2.821207,0.352931,-0.118718,-0.465304,0.224041,1.018965,-0.259254,0.601721,-0.545098,-0.627180,...,0.730058,-1.207904,-0.321726,-0.111557,0.984355,-0.710246,0.453786,1.451899,0.476924,0.079689
9,-0.134119,1.984342,0.281666,-0.231477,-0.527362,-0.039399,0.666419,-1.748122,-0.672592,-1.471874,...,0.798719,1.204593,1.351075,-0.121728,-0.486702,0.161823,0.437470,-0.140334,-0.268765,-1.459773


# Application of PCA

In [121]:
#PCA (Principal Component Analysis) for Entity Word-Level
pca = PCA(n_components=1)
pca.fit(df1)
x_pca = pca.transform(df1)
print('Dimension Before PCA:',df1.shape,'\n''Dimension After PCA:',x_pca.shape)
x_pca

Dimension Before PCA: (18, 768) 
Dimension After PCA: (18, 1)


array([[ -9.56115624],
       [ -6.19110995],
       [ -5.65536481],
       [ -4.3998992 ],
       [ -3.34260658],
       [ -4.34299499],
       [ -7.84049956],
       [  1.79587812],
       [-10.05822841],
       [  0.53559049],
       [ 11.4499833 ],
       [  3.332994  ],
       [  5.40590355],
       [ 14.81978099],
       [ -2.35781587],
       [ 23.52525846],
       [  2.45149502],
       [ -9.56720834]])

In [150]:
#PCA (Principal Component Analysis) for Sequence-Level
pca = PCA(n_components=1)
pca.fit(x_pca.reshape(1,18)*10)
x_pcaa = pca.transform(x_pca.reshape(1,18))
print('Dimension Before PCA:',df1.shape,'\n''Dimension After PCA:',x_pca.shape)
x_pcaa

Dimension Before PCA: (18, 768) 
Dimension After PCA: (18, 1)


C:\Users\Admin\anaconda3\lib\site-packages\sklearn\decomposition\_pca.py:499: RuntimeWarning:

invalid value encountered in true_divide



array([[86.05040616]])

In [151]:
#PCA (Principal Component Analysis) for Sequence-Level
pca = PCA(n_components=1)
pca.fit(x_pca2.reshape(1,18)*10)
x_pcaa = pca.transform(x_pca2.reshape(1,18))
print('Dimension Before PCA:',df1.shape,'\n''Dimension After PCA:',x_pca.shape)
x_pcaa

Dimension Before PCA: (18, 768) 
Dimension After PCA: (18, 1)


C:\Users\Admin\anaconda3\lib\site-packages\sklearn\decomposition\_pca.py:499: RuntimeWarning:

invalid value encountered in true_divide



array([[-74.29408186]])

In [134]:
ener1 = x_pca[3]
amer1 = x_pca[13]
europe1 = x_pca[15]

In [ ]:
siap1 = {'Variable 1':x_pca[:,0],'Variable 2':x_pca[:,1], 'Variable 3':x_pca[:,2],'Topic':1}
df1_pca = pd.DataFrame(data=siap1, columns=['Variable 1','Variable 2','Variable 3','Topic'])
df1_pca

In [110]:
siap1 = {'Variable 1':x_pca[:,0],'Variable 2':x_pca[:,1],'Topic':1}
df1_pca = pd.DataFrame(data=siap1, columns=['Variable 1','Variable 2','Topic'])

In [123]:
siap1 = {'Variable 1':x_pca[:,0], 'Topic':1}
df1_pca = pd.DataFrame(data=siap1, columns=['Variable 1','Topic'])
df1_pca

,Variable 1,Topic
0,-9.561156,1
1,-6.191110,1
2,-5.655365,1
3,-4.399899,1
4,-3.342607,1
5,-4.342995,1
6,-7.840500,1
7,1.795878,1
8,-10.058228,1
9,0.535590,1


In [135]:
#PCA (Principal Component Analysis) for Entity Word-Level
pca = PCA(n_components=1)
pca.fit(df2)
x_pca2 = pca.transform(df2)
print('Dimension Before PCA:',df2.shape,'\n''Dimension After PCA:',x_pca2.shape)
x_pca2

Dimension Before PCA: (18, 768) 
Dimension After PCA: (18, 1)


array([[ 8.25489798],
       [ 2.71200596],
       [-6.25198875],
       [-7.31118651],
       [-4.46806705],
       [ 2.08382068],
       [-0.50758695],
       [-6.31914621],
       [17.17212384],
       [-7.04977253],
       [-2.09039812],
       [-4.27600973],
       [-2.84005302],
       [-6.61171339],
       [-7.98278371],
       [22.52454036],
       [-5.30315669],
       [ 8.26447384]])

In [136]:
ener2 = x_pca2[5]
amer2 = x_pca2[8]
europe2 = x_pca2[15]

In [137]:
print(ener1, ener2)
print(amer1, amer2)
print(europe1, europe2)

[-4.3998992] [2.08382068]
[14.81978099] [17.17212384]
[23.52525846] [22.52454036]
